# Data wrangling: vector and tabular data

Often, datasets need to go through a series of data wrangling and transformation steps before they are ready for analysis or visualisation tasks. This lab will demonstrate several data wrangling and transformation operations focussing on vector and tabular data. 

We will work with a subset of the AgriFieldNet Competition Dataset <a href="https://mlhub.earth/data/ref_agrifieldnet_competition_v1" target="_blank">(Radiant Earth Foundation and IDinsight, 2022)</a> which has been published to encourage people to develop machine learning models that classify a field's crop type from satellite images. This dataset consists of a series of directories with each directory corresponding to a 256 x 256 pixel Sentinel-2 satellite image footprint and information about field boundaries and crop types within each image. 

This data is subset from a <a href="https://beta.source.coop/radiantearth/agrifieldnet-competition/" target="_blank">larger dataset</a> covering agricultural fields in four Indian states: Odisha, Uttar Pradesh, Bihar, and Rajasthan. The field boundaries and crop type labels were captured by data collectors from IDinsight's Data on Demand team and the satellite image preparation was undertaken by the Radiant Earth Foundation. 

Often, datasets for machine learning computer vision tasks (e.g. see the datasets on Radiant Earth's <a href="https://beta.source.coop/radiantearth/agrifieldnet-competition/" target="_blank">Source Cooperative</a>) are provided with data samples for model development spread across many sub-directories. Prior to model training you need to extract the data from these directories and assemble it in a way that it can be passed into a model. The process of transforming data to a format ready for machine learning model development is called **feature engineering**. This lab will demonstrate how to convert the image format data into a vector-tabular dataset where each row corresponds to a field with columns for spectral reflectance measured by the Sentinel-2 sensor and a label for the crop type of the field. This dataset can then be used for machine learning tasks to predict what crop is growing in a field using satellite images, specifically relating patterns of spectral reflectance to a crop type. In this lab you will use the following data wrangling operations:

* *zonal statistics* using raster and vector data to compute mean Sentinel-2 spectral reflectance within each field boundary.
* *combining datasets* by stacking columns and rows to create larger tables and implementing spatial and non-spatial joins.
* *subsetting operations* to extract data from tabular `DataFrame`s.
* *geometry operations* to reproject and change the shape of geometries representing field locations. 

## Setup

### Run the labs

You can run the labs locally on your machine or you can use cloud environments provided by Google Colab. **If you're working with Google Colab be aware that your sessions are temporary and you'll need to take care to save, backup, and download your work.**

<a href="https://colab.research.google.com/github/geog3300-agri3003/coursebook/blob/main/docs/notebooks/week-4_2.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Download data

If you need to download the data for this lab, run the following code snippet.

In [ ]:
import os
import subprocess

if "data_lab-4_2" not in os.listdir(os.getcwd()):
    subprocess.run('wget "https://github.com/geog3300-agri3003/lab-data/raw/main/data_lab-4_2.zip"', shell=True, capture_output=True, text=True)
    subprocess.run('unzip "data_lab-4_2.zip"', shell=True, capture_output=True, text=True)
    if "data_lab-4_2" not in os.listdir(os.getcwd()):
        print("Has a directory called data_lab-4_2 been downloaded and placed in your working directory? If not, try re-executing this code chunk")
    else:
        print("Data download OK")

### Install packages

If you're working in Google Colab, you'll need to install the required packages that don't come with the colab environment.

In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install xarray[complete]
    !pip install rioxarray
    !pip install mapclassify
    !pip install rasterio
    !pip install rasterstats

## Quick recap: What is data wrangling?

<a href="https://r4ds.had.co.nz/wrangle-intro.html" target="_blank">Wickham and Grolemund (2017)</a> and <a href="https://wesmckinney.com/book/" target="_blank">McKinney (2022)</a> state that data wrangling consists of data import, data cleaning, and data transformation. 

#### Data import

Data import was covered in week 3 with examples of how to read tabular, vector, and raster data into Python programs. 

#### Data cleaning

Data cleaning includes handling outliers and missing data. Here, we'll cloud mask Sentinel-2 remote sensing images, which is a data cleaning exercise. 

#### Data transformation

<a href="https://wesmckinney.com/book/" target="_blank">McKinney (2022)</a> define data transformation as mathematical or statistical operations applied to data to generate new datasets. Data transformation can also include operations that reshape datasets or combine two or more datasets.

### Import modules

In [ ]:
import os
import time

import rioxarray as rxr
import xarray as xr
import plotly.express as px
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors

from rasterstats import zonal_stats

import plotly.io as pio

# setup renderer
if 'google.colab' in str(get_ipython()):
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "jupyterlab"

In [ ]:
data_path = os.path.join(os.getcwd(), "data_lab-4_2")

### Lab data

Let's quickly inspect the files that we have downloaded for this lab.

In [ ]:
os.listdir(data_path)

* *agrifieldnet_field_ids.parquet* is a geospatial file storing the field boundaries as a polygon geometry and an integer field id.
* *raster_labels.tif* is a GeoTIFF file where each pixel value indicates what crop was grown in that pixel.
* *s2_reflectance_agrifieldnet_competition_v1_source_0a664.tif* is a multispectral <a href="https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED" target="_blank">Sentinel-2 image covering the extent of the fields.

Let's start by exploring the Sentinel-2 image data. If we visualise it as an RGB image it should look like it covers and agricultural region.  

In [ ]:
s2_path = os.path.join(data_path, "s2_reflectance_agrifieldnet_competition_v1_source_0a664.tif")
s2 = rxr.open_rasterio(s2_path)

In [ ]:
s2

In [ ]:
s2.sel(band=[4, 3, 2]).plot.imshow(robust=True)

#### Recap quiz

**Why have we used the `sel()` method to select the bands 4, 3, and 2 in that order?**

<details>
    <summary><b>answer</b></summary>

Bands 4, 3, and 2 correspond to reflectance in the red, green, and blue wavelengths. Subsetting these bands allows us to render the image as an RGB true colour composite on our display. 
</details>

Next, let's explore data in the `raster_labels.tif` file. This should be a GeoTIFF file storing raster data with the same projection system and x and y dimensions as the satellite image. Each pixel is assigned a numeric value that corresponds to a crop type. Based on the dataset's documentation, this is the mapping between numeric values and crop types in the labels dataset. 

* 1 - Wheat
* 2 - Mustard
* 3 - Lentil
* 4 - No crop/Fallow
* 5 - Green pea
* 6 - Sugarcane
* 8 - Garlic
* 9 - Maize
* 13 - Gram
* 14 - Coriander
* 15 - Potato
* 16 - Bersem
* 36 - Rice

In [ ]:
labels_path = os.path.join(data_path, "raster_labels.tif")
labels = rxr.open_rasterio(labels_path)
labels.sel(band=1).plot.imshow(cmap="Set2")

#### Recap quiz

**Can you check that the `xarray.DataArray` object storing the crop type labels has the same coordinate reference system (CRS) and X and Y dimensions as the `xarray.DataArray` object storing the satellite image?**

In [ ]:
## ADD CODE HERE

<details>
    <summary><b>answer</b></summary>

```python
print(f"The shape of the s2 array is {s2.shape}")
print(f"The shape of the crop type labels array is {labels.shape}")
print("")
print(f"The CRS of the s2 array is {s2.rio.crs}")
print(f"The shape of the crop type labels array is {labels.rio.crs}")
```
</details>

Finally, let's read in the field boundaries and display them on a web map. 

In [ ]:
gdf_field_ids = gpd.read_parquet(os.path.join(data_path, "agrifieldnet_field_ids.parquet"))

In [ ]:
gdf_field_ids.explore()

## Zonal statistics

We can use the `zonal_stats()` function from the <a href="https://pythonhosted.org/rasterstats/manual.html#statistics" target="_blank">rasterstats package</a> to perform zonal statistics in Python. Zonal statistics summarise raster pixel values within zones. Here, we're going to compute the average spectral reflectance values recorded in our Sentinel-2 image dataset for each of the fields that we have a crop type label for. 

The `zonal_stats()` function takes in vector data of geometries (zones) that summary statistics are computed for as its first argument (or a file path to a vector file), the path to a raster dataset (i.e. a GeoTIFF file) as its second argument, a band argument that specifies which band of the raster data to compute zonal statistics for, and optionally a list of statistics to compute to the `stats` argument. You can find the full list of parameters for the `zonal_stats()` function in the package documentation.

**Can you look up what the `all_touched` parameter of the `zonal_stats()` function is used for in the <a href="https://pythonhosted.org/rasterstats/manual.html#rasterization-strategy" target="_blank">rasterstats docs</a>?**

Let's write a small routine that will loop over each band in our `xarray.DataArray` object `s2`, compute the zonal statistics for that band, and convert the result to a pandas `DataFrame` object.

In [ ]:
zstats_s2_tmp = []
s2_bands = list(range(1, 13))

for b in s2_bands:
    zstats = zonal_stats(gdf_field_ids, s2_path, stats=["mean"], band=b)
    zstats_s2_tmp.append(pd.DataFrame(zstats))

`zstats_tmp` is a list of pandas `DataFrame`s. Each `DataFrame` stores the mean spectral reflectance values for the fields for separate spectral bands. Let's look at the zonal statistics results for the first band. 

In [ ]:
zstats_s2_tmp[0]

The zonal statistics operation has returned `None` for every field. 

#### Recap quiz

**Can you explore the `xarray.DataArray` object `s2` and the `GeoDataFrame` object `gdf_field_ids` to see why our zonal statistics operation is returning `None`?**

**Tip: for the zonal statistics operation to work, the raster and vector data need to be aligned in space. Consider checking the CRS of the datasets, and, if necessary changing the CRS of the `GeoDataFrame` `gdf_field_ids` to see if that helps.**

In [ ]:
## ADD CODE HERE

<details>
    <summary><b>answer</b></summary>

For the zonal statistics operation to work, you will need to align and reproject the vector data to match the raster dataset. We can do this by calling the <code>to_crs()</code> method on a <code>GeoDataFrame</code> and passing in the CRS of the raster dataset. This is a geometry transformation operation.

```python
# get the CRS for the s2 DataArray
s2_crs = s2.rio.crs

zstats_s2_tmp = []
s2_bands = list(range(1, 13))

## Look at how we call to_crs() on gdf_field_ids
for b in s2_bands:
    zstats = zonal_stats(gdf_field_ids.to_crs(s2_crs), s2_path, stats=["mean"], band=b)
    zstats_s2_tmp.append(pd.DataFrame(zstats))

zstats_s2_tmp[0]
```
</details>

We also need to convert the crop type labels into a tabular dataset where the crop type of each field is recorded. We can do this using a zonal statistics operation, but we do not want to use the mean as our summarising function. 

#### Recap quiz

**Can you compute the zonal statistics for each field using the crop type labels GeoTIFF file referenced by `labels_path` and using the majority function to summarise the raster data within zones?**

**Refer to the <a href="https://pythonhosted.org/rasterstats/manual.html#zonal-statistics" target="_blank">rasterstats</a> docs for information on selecting the summarising function for the zonal statistics operation.**

In [ ]:
## ADD CODE HERE

<details>
    <summary><b>answer</b></summary>

```python
zstats = zonal_stats(gdf_field_ids.to_crs(s2.rio.crs), labels_path, band=1, stats=["majority"])
labels_df = pd.DataFrame(zstats)
```
</details>

## Combining datasets

Often, we have to deal with datasets that are spread across files or objects in our programs and we need to <a href="https://wesmckinney.com/book/data-wrangling#prep_merge_join" target="_blank">combine</a> them into one dataset for analysis. There are two common techniques for combining datasets: concatenating (stacking) or joining.

Concatenating or stacking datasets is the process of appending data items (e.g. adding new columns or rows to an existing `DataFrame`). 

Joins refer to combining datasets based on the relationships between the two dataset. Key-based / tabular joins combine datasets by matching on common variables. Spatial joins combine datasets by matching datasets based on feature's relationships in space. 

### Concatenating / stacking datasets

The variable `zstats_s2_tmp` should reference a list of `DataFrame` objects, where each `DataFrame` stores the zonal statistics results for a spectral band. However, we need to combine the separate `DataFrame` objects into one `DataFrame`. We can do this by concatenating (stacking) the `DataFrame`'s along an axis. Let's inspect the first `DataFrame` in `zstats_s2_tmp`.

In [ ]:
zstats_s2_tmp[0]

We can see that there is a single column of data which represents the mean spectral reflectance values for band 1 for each of the nine fields. We can combine this data with the `GeoDataFrame` storing the field id and boundary geometry using the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html#pandas.concat" target="_blank">pandas `concat()` function</a>. `concat()` takes a list of `DataFrame`s as an argument and an axis to stack the data items along; here, we want to append columns so we stack along the second axis (axis 1 - remember Python indexes from 0).

In [ ]:
# first, let's concatenate all the zonal statistics results for the Sentinel-2 bands
s2_df = pd.concat(zstats_s2_tmp, axis=1, ignore_index=True)

In [ ]:
# second, let's concatenate the zonal statistics results for the Sentinel-2 bands with the field id and boundaries
gdf_s2 = pd.concat([gdf_field_ids, s2_df], axis=1, ignore_index=True)
gdf_s2.columns = ["field_id", "geometry"] + ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12"] 
gdf_s2.head()

You will note that we updated the column names by assigning a list to the `columns` attribute of the `gdf_s2` `GeoDataFrame`.

#### Recap quiz

**Can you combine the `labels_df` `DataFrame` with the `gdf_field_ids` `GeoDataFrame` using the pandas `concat()` function? Assign the result to the variable `gdf_labels`.**

In [ ]:
## ADD CODE HERE

<details>
    <summary><b>answer</b></summary>

```python
gdf_labels = pd.concat([gdf_field_ids, labels_df], axis=1, ignore_index=True)
gdf_labels.columns = ["field_id", "geometry"] + ["crop_label"]
```
</details>

### Key-based joins

We now have two separate `GeoDataFrame` objects in our program. We have a `GeoDataFrame` storing average spectral reflectance values for each field referenced by `gdf_s2` and a `GeoDataFrame` storing the crop type label for each field in `gdf_labels`. There is a matching column in both of these `GeoDataFrame`s - `field_id`.

When two tables have a matching column(s) we can use join operations to merge them. Rows in both tables are matched using common values in the matching column(s) and the joined table has columns from both tables. 

Joining tables is a common operation in relational databases using SQL and the same operations can be implemented in Pandas using the <a href="https://pandas.pydata.org/docs/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging" target="_blank">`merge()`</a> functions. 

Some important concepts for join operations:

* The columns with values used to match rows are called **keys**.
* **one-to-one** joins are where there is exactly one match between rows in the two tables being joined.
* **many-to-one** joins are where a row in one table can match one or more rows in another table.
* **left joins** keep all rows in the left table and only matching rows in the right table. 
* **inner joins** keep only matching rows in the left and right tables. 

The Pandas <a href="https://pandas.pydata.org/docs/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging" target="_blank">`merge()`</a> docs and <a href="https://wesmckinney.com/book/data-wrangling.html#prep_merge_join" target="_blank">McKinney (2022)</a> provide useful explanations for how join operations work.

![](https://github.com/data-analysis-3300-3003/figs/raw/main/week-4-joins.jpg)

Let's consider these concepts in the context of joining our `GeoDataFrame` of each field's spectral reflectance values with a `GeoDataFrame` of each field's crop type label. 

The matching column in both tables is `field_id`. This the joining key. 

We are joining the two tables on `field_id` which should be unique to each field. Therefore, we are implementing a one-to-one join. We only want to keep fields where there is a crop type value and spectral reflectance values. Therefore, we'll use an inner join.

Pandas `merge()` function can take the following arguments:

* `left` - left table in the join.
* `right` - right table in the join.
* `how` - whether to use a left or inner join.
* `left_on` - columns in left table to use as keys.
* `right_on` - columns in the right table to use as keys.

As there is a `geometry` column in both `GeoDataFrame`s, let's drop it from `gdf_s2` to avoid it being duplicated in the joined dataset.

In [ ]:
df_s2 = gdf_s2.drop(columns=["geometry"])

Review the <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html" target="_blank">pandas `merge()`</a> documentation to see how we're setting up our join. The `how` argument specifies the type of join (`"inner"` means we only want to keep rows in both datasets where there is a match on the joining column) and the column to join on (`"field_id"` is the common column in both datasets that we're joining on).  

In [ ]:
gdf_joined = gdf_labels.merge(df_s2, how="inner", on="field_id")

#### Recap quiz

**Can you check that our join has been completed correctly? Think about what the `DataFrame` should look like after a completed join and how you can inspect the `DataFrame` to check this.** 

In [ ]:
## ADD CODE HERE

<details>
    <summary><b>answer</b></summary>

You could print out the columns of the joined dataset to check that we have columns corresponding to the crop type label and the spectral reflectance bands. `GeoDataFrame`s and `DataFrame`s have a `columns` attribute that stores the column names as a `list`. 

```python
print(gdf_joined.columns)
```
<p></p>

You could also print out the first few rows of the dataset to check it looks sensible. The `DataFrame`'s `head()` method does this.

```python
gdf_joined.columns()
```
</details>

## Spatial Joins

Spatial join operations join the attributes of two vector layers based on their relationship in space. For example, if we have a `GeoDataFrame` storing field boundaries (polygon geometries) and field attributes and another `GeoDataFrame` storing district boundaries (polygon geometries) and a district name as an attribute, we can join the the two tables based on the largest intersection (overlap) between field boundaries and district boundaries. If the field boundaries `GeoDataFrame` was the left table in the spatial join, for each row (or geometry feature) the district name from the district with largest intersection would be joined to that table in a new column. 

GeoPandas provides an <a href="https://geopandas.org/en/stable/docs/user_guide/mergingdata.html#spatial-joins" target="_blank">`sjoin()` function</a> that can be used for spatial joins of two `GeoDataFrames`. The `sjoin()` function expects the following as arguments:

* `left_df` - left `GeoDataFrame` in the spatial join.
* `right_df` - right `GeoDataFrame` in the spatial join - columns from the `right_df` will be joined to `left_df`.
* `how` - whether to use a left, inner, or right join.
* `predicate` - a binary predicate that defines the spatial relationship between features in `right_df` and `left_df`. 

Binary predicates that can be used are:

* intersects
* contains
* crosses
* within
* touches
* overlaps

Intersects is the default predicate for spatial joins in GeoPandas. 

![](https://github.com/data-analysis-3300-3003/figs/raw/main/week-4-spatial-join.jpg)

It would be useful to include some information about where in India the fields we're working with are located. To do this can perform a spatial join to combine our field boundaries with the boundaries of administrative units in India, districts in this case.

We need to read in district geometries for India obtained from <a href="https://www.geoboundaries.org" target="_blank">geoBoundaries</a>. 

In [ ]:
india_districts = gpd.read_file(os.path.join(os.getcwd(), "data_lab-4_2", "geoBoundaries-IND-ADM2_simplified.topojson"))
india_districts.head()

### Subsetting pandas `DataFrame`s

To <a href="https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#how-do-i-select-a-subset-of-a-dataframe" target="_blank">subset data from a pandas `DataFrame`</a> we use the square brackets `[]` to specify the data values we'd like to extract. This is an example of non-spatial subsetting.

**Selection by labels**

*Selection by labels* refers to selecting values from a `DataFrame` by their label (i.e. column name).

* To select a column from a `DataFrame` we pass the column name into `[]` (e.g. `india_districts = india_districts["shapeName"]` where `shapeName` is a `Series` object).
* To select many columns from a `DataFrame` we pass a list of column names into `[]` (e.g. (e.g. `india_districts = india_districts[["shapeName", "geometry"]]`).

**Selection by position**

*Selection by position* refers to selecting values from a `DataFrame` by their index position (i.e. row or column number starting at 0).

* To select the $n^{th}$ row pass in `[n-1:n]`. Remember that Python indexes from zero so the $n-1$ index position is the $n^{th}$ row. The slice operator `:` is exclusive so `[n-1:n]` will only select the row at `n-1` (e.g. to select the $2^{nd}$ row use `df_row_2 = gdf[1:2]`).
* To select a slice of rows use the slice operator (e.g. to select the first 10 rows use `df_10_rows = gdf[0:10]`).

**Selection by condition**

*Selection by condition* selects rows that are `True` based on a condition (e.g. selecting all rows with a `shapeArea` greater than 1 - `india_districts_gt1 = india_districts[india_districts["shapeArea"] > 1]`).

**`loc[]` and `iloc[]`**

The more robust approach to subsetting data from `DataFrame`s is using the <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#different-choices-for-indexing" target="_blank">`loc` and `iloc` methods</a>. 

* `loc` is used for selecting by labels (e.g. `india_districts = india_districts.loc[:, ["shapeName", "geometry"]]` - note the `[:, ["shapeName", "geometry"]]` syntax where `:` means select all rows).
* `iloc` is used for selecting by position (e.g. to select the first 10 rows use `df_10_rows = gdf.iloc[0:10, :]`).
* To select the first 10 rows and columns we'd use `df_10_rows_10_cols = gdf.iloc[0:10, 0:10]`).

#### Recap quiz

<details>
    <summary><b>How many rows will the object referenced by <code>df_temp</code> have after calling <code>df_temp=df.iloc[0:5, :]</code>?</b></summary>
5 rows at index positions 0 to 4 from the <code>DataFrame</code> <code>df</code>.
</details>

<p></p>

<details>
    <summary><b>How many columns from <code>df</code> will the object referenced by <code>df_temp</code> have after calling <code>df_temp=df.iloc[0:5, :]</code>?</b></summary>
All the columns from <code>df</code>.
</details>

<p></p>

**Use `loc` and the `GeoDataFrame` referenced by `india_districts` to select the `shapeName` and `geometry` columns. Assign the result to the variable `india_districts`.**

In [ ]:
## ADD CODE HERE

<details>
    <summary><b>answer</b></summary>

```python
india_districts = india_districts.loc[:, ["shapeName", "geometry"]]
```
</details>

Next, let's tidy up our subsetted dataset by giving it neat column names and setting the CRS.

In [ ]:
india_districts.columns = ["district", "geometry"]
india_districts = india_districts.set_crs("EPSG:4326")
india_districts.head()

In [ ]:
india_districts.plot(column="district")

Now we can implement our spatial join. We use the `sjoin()` function from `GeoPandas` and specify the two datasets we are spatially joining. Again, we use an inner join where we keep only rows in the left and right `GeoDataFrame`s where there is a match in space. Here, we determine a match in space where the polygon in the left `GeoDataFrame` intersects with a polygon in the right `GeoDataFrame`.

In [ ]:
gdf_district = gpd.sjoin(
    left_df=gdf_joined, 
    right_df=india_districts, 
    how="inner", 
    predicate="intersects"
)
gdf_district.head()

We now have a clean and tidy tabular dataset ready for analysis (e.g. building machine learning models to classify crop type based on spectral reflectance). 

#### Recap quiz

**If you were to expand this workflow to be handle a large number of fields across many satellite image scenes, what Python tools might help you automate this process?**

<details>
    <summary><b>answer</b></summary>

For loops. You could include this data transformation process inside a for loop and iterate over a sequence of satellite images and converting them to tabular datasets. 
</details>